In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
df1 = pd.read_csv('dataset/MovementAAL/dataset/MovementAAL_RSS_1.csv')
df2 = pd.read_csv('dataset/MovementAAL/dataset/MovementAAL_RSS_2.csv')

In [5]:
path = 'dataset/MovementAAL/dataset/MovementAAL_RSS_'
sequences = list()
for i in range(1,315):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)

dataset/MovementAAL/dataset/MovementAAL_RSS_1.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_2.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_3.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_4.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_5.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_6.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_7.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_8.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_9.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_10.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_11.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_12.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_13.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_14.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_15.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_16.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_17.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_18.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_19.csv
dataset/MovementAAL/dataset/MovementAAL_

dataset/MovementAAL/dataset/MovementAAL_RSS_194.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_195.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_196.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_197.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_198.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_199.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_200.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_201.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_202.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_203.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_204.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_205.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_206.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_207.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_208.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_209.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_210.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_211.csv
dataset/MovementAAL/dataset/MovementAAL_RSS_212.csv
dataset/Move

In [6]:
targets = pd.read_csv('dataset/MovementAAL/dataset/MovementAAL_target.csv')
targets = targets.values[:,1]

In [7]:
groups = pd.read_csv('dataset/MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]

In [8]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

In [10]:
to_pad = 129
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from tensorflow.keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [11]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

In [12]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [13]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               267264    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________


In [19]:
train.shape, train_target.shape

((106, 60, 4), (106,))